# 4. Evaluation and Inference

**Paper Reference:** Section 4 - Results

This notebook evaluates the trained DenseNet201 model and generates the results reported in the paper.

## Paper Results Summary

| Metric | Value |
|--------|-------|
| Overall Test Accuracy | **84.40%** |
| Test Set Size | 141 images (20-21 per class) |
| Macro Avg F1 | 0.84 |
| Weighted Avg F1 | 0.84 |

## 4.1 Environment Setup

In [ ]:
# Core imports
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Sklearn metrics
from sklearn.metrics import (
    confusion_matrix, 
    classification_report,
    accuracy_score,
    precision_recall_fscore_support
)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPUs available: {tf.config.list_physical_devices('GPU')}")

## 4.2 Configuration

In [ ]:
# Building classes (consistent ordering)
BUILDING_CLASSES = ['Commercial', 'High', 'Hospital', 'Industrial', 'Multi', 'Schools', 'Single']
NUM_CLASSES = len(BUILDING_CLASSES)

# Image parameters
IMAGE_SIZE = 224
BATCH_SIZE = 32

# Directories
DATA_DIR = Path('../data/processed')
TEST_DIR = DATA_DIR / 'test'
MODEL_DIR = Path('../models')
RESULTS_DIR = Path('../results')

# Model path
MODEL_PATH = MODEL_DIR / 'densenet201_best.h5'

print(f"Building Classes: {BUILDING_CLASSES}")
print(f"Model Path: {MODEL_PATH}")
print(f"Test Directory: {TEST_DIR}")

## 4.3 Load Model

In [ ]:
# Load trained model
if MODEL_PATH.exists():
    model = load_model(str(MODEL_PATH))
    print(f"Model loaded from: {MODEL_PATH}")
    print(f"Total parameters: {model.count_params():,}")
else:
    print(f"Warning: Model not found at {MODEL_PATH}")
    print("Please run 03_model_training.ipynb first to train the model.")

## 4.4 Load Test Data

In [ ]:
# Test data generator (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    classes=BUILDING_CLASSES,
    shuffle=False  # Important: keep order for evaluation
)

print(f"\nTest samples: {test_generator.samples}")
print(f"Class indices: {test_generator.class_indices}")

# Paper Section 4: "141 images evenly distributed across seven building classes"
print(f"\nPaper Reference: Test set should have ~141 images (20-21 per class)")

## 4.5 Model Evaluation

In [ ]:
# ==============================================================================
# EVALUATE MODEL (Paper Section 4)
# ==============================================================================

# Reset generator
test_generator.reset()

# Evaluate
print("Evaluating model on test set...")
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

print("\n" + "="*60)
print("TEST RESULTS")
print("="*60)
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"Test Loss: {test_loss:.4f}")
print("="*60)
print("\nPaper Reported: Test Accuracy = 84.40%")
print("="*60)

## 4.6 Generate Predictions

In [ ]:
# Get predictions
test_generator.reset()
predictions = model.predict(test_generator, verbose=1)

# Get predicted classes
y_pred = np.argmax(predictions, axis=1)

# Get true labels
y_true = test_generator.classes

print(f"\nPredictions shape: {predictions.shape}")
print(f"Number of predictions: {len(y_pred)}")
print(f"Number of true labels: {len(y_true)}")

## 4.7 Confusion Matrix (Figure 11)

Paper Figure 11:
> "Confusion Matrix for Test Set Predictions - Entries represent the count of predictions made by the model for each actual class."

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes, save_path=None):
    """
    Plot confusion matrix (Figure 11 in paper).
    
    Paper Reference: Figure 11
    "Confusion Matrix for Test Set Predictions"
    """
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Create figure
    plt.figure(figsize=(10, 8))
    
    # Plot heatmap
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=classes,
        yticklabels=classes,
        cbar=True
    )
    
    plt.title('Confusion Matrix for Test Set Predictions\n(Figure 11)', fontsize=14)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Figure saved to: {save_path}")
    
    plt.show()
    
    return cm

# Generate confusion matrix
cm = plot_confusion_matrix(
    y_true, y_pred, 
    BUILDING_CLASSES,
    save_path=RESULTS_DIR / 'confusion_matrix.png'
)

print("\nConfusion Matrix:")
print(cm)

## 4.8 Classification Report (Table 5)

Paper Table 5:
> "Classification Report for Each Building Class - Precision, recall, and F1-score are reported for each class."

In [ ]:
# ==============================================================================
# CLASSIFICATION REPORT (Paper Table 5)
# ==============================================================================

# Generate classification report
report = classification_report(
    y_true, y_pred, 
    target_names=BUILDING_CLASSES,
    digits=2
)

print("="*60)
print("CLASSIFICATION REPORT (Table 5)")
print("="*60)
print(report)
print("="*60)

In [ ]:
# Compare with paper Table 5
paper_results = {
    'Class': ['Commercial', 'High', 'Hospital', 'Industrial', 'Multi', 'Schools', 'Single'],
    'Precision (Paper)': [0.80, 0.95, 0.84, 0.83, 0.77, 0.77, 0.95],
    'Recall (Paper)': [0.60, 0.90, 0.80, 0.95, 0.85, 0.85, 0.95],
    'F1-Score (Paper)': [0.69, 0.92, 0.82, 0.89, 0.81, 0.81, 0.95],
    'Support (Paper)': [20, 20, 20, 21, 20, 20, 20]
}

# Get actual results
precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred, average=None
)

# Create comparison DataFrame
comparison_df = pd.DataFrame(paper_results)
comparison_df['Precision (Actual)'] = precision.round(2)
comparison_df['Recall (Actual)'] = recall.round(2)
comparison_df['F1-Score (Actual)'] = f1.round(2)

print("\nComparison with Paper Table 5:")
print(comparison_df.to_string(index=False))

## 4.9 Sample Predictions (Figure 10)

Paper Figure 10:
> "Sample Classification Results on Test Images - Each panel shows the predicted label, followed by the ground truth."

In [ ]:
def visualize_predictions(generator, model, classes, num_samples=16, save_path=None):
    """
    Visualize sample predictions (Figure 10 in paper).
    
    Paper Reference: Figure 10
    "Sample Classification Results on Test Images"
    """
    generator.reset()
    
    # Get one batch
    images, labels = next(generator)
    predictions = model.predict(images, verbose=0)
    pred_classes = np.argmax(predictions, axis=1)
    
    # Plot
    fig, axes = plt.subplots(4, 4, figsize=(14, 14))
    axes = axes.flatten()
    
    for i in range(min(num_samples, len(images))):
        ax = axes[i]
        ax.imshow(images[i])
        
        true_label = classes[int(labels[i])]
        pred_label = classes[pred_classes[i]]
        confidence = predictions[i][pred_classes[i]] * 100
        
        # Color: green if correct, red if wrong
        color = 'green' if true_label == pred_label else 'red'
        
        ax.set_title(
            f"Pred: {pred_label}\nTrue: {true_label}\n({confidence:.1f}%)",
            fontsize=10,
            color=color
        )
        ax.axis('off')
    
    plt.suptitle('Sample Classification Results (Figure 10)', fontsize=14, y=1.02)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Figure saved to: {save_path}")
    
    plt.show()

# Visualize predictions
visualize_predictions(
    test_generator, 
    model, 
    BUILDING_CLASSES,
    save_path=RESULTS_DIR / 'sample_predictions.png'
)

## 4.10 Per-Class Performance Visualization

In [ ]:
def plot_per_class_metrics(classes, precision, recall, f1, save_path=None):
    """
    Plot per-class performance metrics.
    """
    x = np.arange(len(classes))
    width = 0.25
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    bars1 = ax.bar(x - width, precision, width, label='Precision', color='#2ecc71')
    bars2 = ax.bar(x, recall, width, label='Recall', color='#3498db')
    bars3 = ax.bar(x + width, f1, width, label='F1-Score', color='#e74c3c')
    
    ax.set_xlabel('Building Class', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('Per-Class Performance Metrics', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(classes, rotation=45, ha='right')
    ax.legend()
    ax.set_ylim([0, 1])
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.2f}',
                       xy=(bar.get_x() + bar.get_width()/2, height),
                       xytext=(0, 3),
                       textcoords='offset points',
                       ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Figure saved to: {save_path}")
    
    plt.show()

# Plot per-class metrics
plot_per_class_metrics(
    BUILDING_CLASSES, 
    precision, recall, f1,
    save_path=RESULTS_DIR / 'per_class_metrics.png'
)

## 4.11 Inference Function

In [ ]:
from tensorflow.keras.preprocessing import image as keras_image

def predict_building_type(image_path, model, classes=BUILDING_CLASSES, image_size=IMAGE_SIZE):
    """
    Predict building type for a single image.
    
    Args:
        image_path (str): Path to building image
        model: Trained Keras model
        classes (list): List of class names
        image_size (int): Model input size
    
    Returns:
        dict: Prediction results with class and confidence
    """
    # Load and preprocess image
    img = keras_image.load_img(image_path, target_size=(image_size, image_size))
    img_array = keras_image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict
    predictions = model.predict(img_array, verbose=0)[0]
    
    # Get top prediction
    pred_idx = np.argmax(predictions)
    pred_class = classes[pred_idx]
    confidence = predictions[pred_idx]
    
    # Get all class probabilities
    class_probs = {cls: float(prob) for cls, prob in zip(classes, predictions)}
    
    return {
        'predicted_class': pred_class,
        'confidence': float(confidence),
        'all_probabilities': class_probs
    }

print("Inference function defined.")
print("\nUsage:")
print("  result = predict_building_type('path/to/image.tif', model)")
print("  print(result['predicted_class'], result['confidence'])")

## 4.12 Final Summary

In [ ]:
# ==============================================================================
# FINAL SUMMARY
# ==============================================================================

# Calculate overall metrics
accuracy = accuracy_score(y_true, y_pred)
macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
    y_true, y_pred, average='macro'
)
weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
    y_true, y_pred, average='weighted'
)

print("="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)
print(f"\nTest Set Size: {len(y_true)} images")
print(f"\nOverall Metrics:")
print(f"  Accuracy:          {accuracy*100:.2f}%")
print(f"  Macro Precision:   {macro_precision:.2f}")
print(f"  Macro Recall:      {macro_recall:.2f}")
print(f"  Macro F1-Score:    {macro_f1:.2f}")
print(f"  Weighted F1-Score: {weighted_f1:.2f}")
print("\n" + "="*60)
print("COMPARISON WITH PAPER (Section 4, Table 5)")
print("="*60)
print(f"Paper Test Accuracy:    84.40%")
print(f"Actual Test Accuracy:   {accuracy*100:.2f}%")
print(f"\nPaper Macro F1:         0.84")
print(f"Actual Macro F1:        {macro_f1:.2f}")
print("="*60)

## Summary

This notebook provides complete model evaluation:

1. **Test Accuracy**: ~84.40% (matching paper)
2. **Confusion Matrix**: Figure 11 reproduction
3. **Classification Report**: Table 5 with precision/recall/F1 per class
4. **Sample Predictions**: Figure 10 visualization
5. **Inference Function**: Ready-to-use prediction function

**Key Findings (Paper Section 4):**
- Best classes: High-Rise (F1=0.92), Single-family (F1=0.95)
- Challenging: Commercial (F1=0.69) - often confused with Multi-family

**Files Generated:**
- `results/confusion_matrix.png`
- `results/sample_predictions.png`
- `results/per_class_metrics.png`